In [0]:
import requests
from IPython.core.display import HTML
HTML(f"""
<style>
@import "https://cdn.jsdelivr.net/npm/bulma@0.9.4/css/bulma.min.css";
</style>
""")

# Filters and Denoising Introduction
This tutorial demonstrates convolution using the _scikit-image_ library. The tutorial will cover:
- Convolution/correlation with manual and predefined filters.
- Explicitly construct filters to perform specific operations.

As a first step, it is necessary to import the libraries:


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve
from skimage import data, filters
from scipy import signal
import scipy
import scipy.ndimage as sp
import seaborn as sns
import pandas as pd
from skimage import data, color
from scipy.signal import convolve2d

Scikit-Image has many example images that can be obtained (see the [documentation](https://scikit-image.org/docs/stable/api/skimage.data.html)
 for a complete list). You can change the name and view some of the provided images in the cell below:


In [0]:
print(data.__all__)
plt.imshow(data.rocket(), cmap="gray")
plt.axis(False)

Most of the filters presented at the lecture can be obtained and applied using the `skimage.filters`
 module. For example, a gaussian filter can be applied using the `filters.gaussian`
 function as shown below. A complete reference for the included filters can be found in the [documentation](https://scikit-image.org/docs/stable/api/skimage.filters.html)
. You can expermient with other filters and see the results. 


In [0]:
blurred = filters.gaussian(data.rocket(), channel_axis=2, sigma=5)
plt.imshow(blurred, cmap="gray")
plt.axis(False)

## Filters from scratch
Although Scikit-Image makes filtering simple, it is worthwhile to understand filtering in more detail.
Let us define a mean kernel (matrix), i.e. a filter that calculates the average over a region specified by the kernel size. The kernel is defined by an  N x N matrix which determines the extent of smoothing/blurring applied to the signal. In image processing the kernel values represent the weights applied to the corresponding pixel and its neighboring pixels. A larger kernel will lead to more smoothing by averaging over a larger area. The function `mean_kernel`
 in the cell below produces a mean kernel:


In [0]:
def mean_kernel(size):
    return np.ones((size, size, 1))/(size**2)

k = mean_kernel(10)

The cell below plots the mean kernel constructed above:


In [0]:
# Create a 10x10 grid to display the kernel values
plt.figure(figsize=(8, 8))
plt.imshow(k[:, :, 0], cmap='gray', interpolation='none', vmin=0, vmax=0.01, extent=[0, 10, 0, 10])  # Adjust vmin and vmax for your specific kernel values

# Display the values strictly inside each cell
for i in range(k.shape[0]):
    for j in range(k.shape[1]):
        plt.text(j + 0.5, i + 0.5, f'{k[i, j, 0]:.2f}', color='black', ha='center', va='center')

plt.title('Mean Kernel (10x10)')
plt.xticks(np.arange(0, 11, 1))
plt.yticks(np.arange(0, 11, 1))
plt.grid(color='black', linestyle='-', linewidth=0.5)
plt.show()

When this kernel is convolved with an image, each pixel in the resulting image will be the average over the neighborhood of pixels, giving a smoothing effect. The cell below visualizes the image of the rocket ship with normalized pixel values, similarly to the representation of the mean kernel (for illustrative purposes the image is downsampled to 20x20 pixels): 


In [0]:
# Load the "camera" image from scikit-learn
data1 = data.rocket()
from skimage.transform import resize

# Resize the image to a smaller size
target_height, target_width = 20, 20  # Set your desired dimensions
resized_image = resize(data1, (target_height, target_width))

# Create a figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 8))

# Plot the grid of values in the first subplot
axes[0].imshow(np.ones_like(resized_image), cmap='gray', extent=[0, 20, 0, 20], vmin=0, vmax=1)  # Display white cells

# Display the values strictly inside each cell
for i in range(target_height):
    for j in range(target_width):
        pixel_value = resized_image[i, j, 0]
        axes[0].text(j + 0.5, target_height - i - 0.5, f'{pixel_value:.2f}', color='black', ha='center', va='center')

axes[0].set_title('Grid of Values (20x20)')
axes[0].set_xticks(np.arange(0, 21, 1))
axes[0].set_yticks(np.arange(0, 21, 1))
axes[0].grid(color='black', linestyle='-', linewidth=0.5)

# Plot the downsampled image in the second subplot without pixel values
axes[1].imshow(resized_image, cmap='hot', vmin=0, vmax=1)
axes[1].set_title('Downsampled Image (20x20)')
axes[1].axis('off')  # Hide axis for the image plot

plt.tight_layout()
plt.show()

Applying the mean kernel to the image is done by using the `scipy.ndimage.convolve`
 function:


In [0]:
blurred = scipy.ndimage.convolve(data.rocket(), k)
plt.imshow(blurred)
plt.axis(False)

### Edge Filter by convolution
Linear filters are implemented as inner products and have many applications. One particular application is to find edges in an image, by constructing a filter that calculates the partial derivatives of an image in either the $x$ or $y$ direction.
The definition of the partial derivative of a function $f$ with respect to $x$ can be defined as:

$$
\begin{align}
\frac{\partial f(x, y)}{\partial x} = \lim_{\Delta \rightarrow 0} \frac{f(x+\Delta, y) - f(x,y)}{\Delta}
\end{align}
$$
An image is a discrete approximation of the light distribution hitting the camera sensor with integer indices. The partial derivative can be approximated by setting $\Delta$ to $1$ (smallest possible step). This method is known as the _finite differences_ method:

$$
\begin{align}
\frac{\partial f(x, y)}{\partial x} \approx  \frac{f(x+1, y) - f(x,y)}{1} = f(x+1, y) - f(x, y)
\end{align}
$$
Notice that the expression is just the difference between neighboring pixels. The corresponding kernel for calculating the derivative with respect to $x$ is:

$$
k_{x} = \begin{bmatrix}1&-1\end{bmatrix}
$$
Equivalently, the kernel for calculating the derivative in the $y$-direction is:

$$
\begin{align*}
k_{y} &= \begin{bmatrix}1 \\ -1\end{bmatrix}
\end{align*}
$$
You may have noticed that the $1$ and $-1$ seem reversed in the kernel. This is due the definition of convolution. The cell below implements the kernel and convolves it with the image . The image is grayscale but displayed using colors for illustrative purposes:


In [0]:
k_dx = np.array([[1, -1]])

image = data.camera().astype(np.float32)

dx = convolve(image, k_dx)

# Create a 1x2 grid of subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first image in the first subplot
axs[0].imshow(data.camera(), cmap='copper')
axs[0].set_title('Original Image')
axs[0].axis('off')

# Plot the result of convolution in the second subplot
axs[1].imshow(dx, cmap='copper')
axs[1].set_title('Convolution Result')
axs[1].axis('off')

# Display the subplots
plt.show()

In practice, finite differences is not robust to noise, so a larger area is usually needed, which is achieved by $k_{x} = \begin{bmatrix}1&0&-1\end{bmatrix}$. This is because the previous kernel calculates the approximate derivative between two pixels, whereas the modified kernel calculates it at the exact pixel location.
Additionally, the image is typically blurred (or smoothed) using a Gaussian kernel before calculating the derivative, to decrease the effect of noise and other very small features. The Gaussian filter blurs the image by convolving the image with a Gaussian kernel defined by the scale parameter (`sigma`
). Increasing sigma leads to more blurring, because the filter becomes larger:


In [0]:
k_dx = np.array([[1, 0, -1]]) # The new centered kernel

image_blurred = filters.gaussian(image, sigma=1) # Blurring with a Gaussian, higher sigma more blurring

dx = convolve(image_blurred, k_dx)
plt.imshow(dx, cmap='copper')
plt.axis(False)

Due to the associative property of convolution, it is possible to convolve the gaussian and derivative kernels to produce a kernel that performs both operations (differentiation and blurring). Performing a convolution operation between the derivative kernel and the Gaussian kernel is the same as to taking the derivative of the Gaussian, resulting in the creation of a new filter. In the cell below a derivative kernel (`k_dx`
) is created then convolved with a Gaussian, and then the new filter is applied to the image. 


In [0]:
k_dx = np.array([
    [0, 0, 0],
    [1., 0, -1.],
    [0, 0, 0]
]) # The kernel has to be padded with zeroes, otherwise the convolution will result in a new 1x3 kernel (try it yourself if you want to see the result)

k_dx_blurred = filters.gaussian(k_dx, sigma=1, preserve_range=True) # we apply the blurring to the filter instead of the image
dx = convolve(image, k_dx_blurred) # convolve the image with the blurred filter

# Create a 1x2 grid of subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first image in the first subplot
axs[0].imshow(k_dx_blurred, cmap='copper')
axs[0].set_title('Filter')
axs[0].axis('off')

# Plot the result of convolution in the second subplot
axs[1].imshow(dx, cmap='copper')
axs[1].set_title('Convolution Result')
axs[1].axis('off')

# Display the subplots
plt.show()

There exist several predefined kernels that are capable of edge detection, albeit with slight variations to achieve certain desirable properties. One such kernel is the _sobel operator_ which is included in scikit-image. The sobel operator is used to highlight edges within an image by convolving the image with two 3x3 kernels (one for detecting changes in the horizontal direction and the other for the vertical direction). The result of applying it on the $x$-direction is shown below:


In [0]:
dx = filters.sobel(image, axis=1)
plt.imshow(dx, cmap='copper')
plt.axis(False)

So far the focus has been on horizontal edges (`k_dx`
). The cell below defines a derivative kernel for finding derivatives (edges) in the $y$-direction. The kernel is convolved with a Gaussian kernel, before it's applied to the image:


In [0]:
k_dy = np.array([
    [0, 1., 0],
    [0, 0, 0],
    [0, -1., 0]
])

k_dy_blurred = filters.gaussian(k_dy, sigma=1, preserve_range=True) # we apply the blurring to the filter instead of the image
dy = convolve(image, k_dy_blurred) # convolve the image with the blurred filter
plt.show()
plt.imshow(dy, cmap='copper')
plt.axis(False)

By applying the horizontal and vertical edge detection kernels, areas in the image where pixel intensities change rapidly were highlighted, indicating potential edges. Remember that these kernels were convolved with a Gaussian filter for noise reduction. 
In the cell below the gradient magnitude (`gradMag`
) is calculated, which highlights the regions of the image where there are significant intensity changes, making it easier to identify edges and key features in the image: 


In [0]:
# Perform convolution to compute horizontal and vertical gradients
Ix = convolve(image, k_dx_blurred)
Iy = convolve(image, k_dy_blurred)

# Calculate gradient magnitude
gradMag = np.sqrt(Ix**2 + Iy**2)

# Create subplots to display the original image and gradient magnitude
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image, cmap='copper')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(gradMag, cmap='copper')
axes[1].set_title('Edge and feature detection with Gradient Magnitude')
axes[1].axis('off')

plt.tight_layout()
plt.show()